<a href="https://colab.research.google.com/github/vishnu-chand/videoMosaic/blob/main/videoMosaic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Photo Mosaic
[Photomosaic](https://en.wikipedia.org/wiki/Photographic_mosaic), is a picture (usually a photograph) that has been divided into (usually equal sized) tiled sections, each of which is replaced with another photograph that matches the target photo. When viewed at low magnifications, the individual pixels appear as the primary image, while close examination reveals that the image is in fact made up of many hundreds or thousands of smaller images

#### Video Mosaic
on the other hand Video Mosaic is simple applying photo mosaic on every frame and ensuring frame consistency  

#### Challenges 
* lets say 
     - image dimension is 720x1280, total frames in video is 1174
     - tile size 32x32
     - number of tile images in tileGallery ~ 200000
* for better visual quality video is scaled up by 4 times and hence the image dimension translate to 2880x5120
* as we are using 32x32 tiles, total grids in single image will be 2880/32x5120/32=14400 grids
* the video of 1175 frames will have 14400x1175=16920000 grids
* all 16920000 grids will replaced by images from tileGallery
* even if we optimally assume, matching one grid with tile image takes ~ 1 ms
    - this will take 0.001x16920000=16920seconds
    - which translate to ~5 hours to complete a video of less than a minute


#### How to solve
* using multiple clustering, dimensionality reduction, activity detection technique total execution time is reduced to ~300 seconds or 4 fps

color feature
* A Tile image is 32x32x3=3072 dimensional vector, we can reduce the dimension using color histogram
* the feature vector color histogram is kept at 125 dimensions

creating tiles
* step1: resize all images tile size eg: 32*32
* step2: calculate 125 dim color histogram feature vector of every image
* step3: using k-means clustering algo, group similar images based on the color feature
* step4: based on population percentile, delete small clusters, this will avoid repeated tiles

photo mosaic
* step1: scale up image to 4x
* step2: split images in 32x32 grids and take color histogram of the grid
* step3: run k-means clustering on grids and group grids into 150 clusters based on color similarity
* step4: map grid cluster to tile cluster based on cosine similarity
* step5: replace images grid with matched tile cluster to create photo mosaic
* step 6: as a final touch up superimpose tiles with image this make global patterns much more visible
video mosaic
* step1: compute euclidean distance between previous and current frame
* step2: if distance is less than certain threshold then re-use previous tiles 
* step3: for the pixels with does not matches with previous frame, compute cluster match as in photo mosaic


In [ ]:
import os
if '' or not os.path.exists('/content/pixUtils2'):
    !pip install youtube_dl
    !rm -rf /content/sample_data
    !git clone https://github.com/vishnu-chand/pixUtils2.git;cd /content/pixUtils2;git checkout 410e006ce8a65c4391540977a6db0d7a7791b1b8
    !mkdir -p /content/db/srcVideos;cd /content/db/srcVideos;youtube-dl -f 'bestvideo[height<=480]+bestaudio/best[height<=480]' https://www.youtube.com/watch?v=lINIOau7MOQ&ab_channel=BryleJhoneGaballo
    !cd /content/db;wget https://cseweb.ucsd.edu/~weijian/static/datasets/celeba/img_align_celeba.zip;unzip img_align_celeba.zip > /dev/null 2>&1

%load_ext autoreload
%autoreload 2

%cd /content/pixUtils2
from pixUtils import *
%cd /content
from scipy.spatial._distance_wrap import cdist_cosine_double_wrap
from sklearn.cluster import MiniBatchKMeans
from pixUtils.threadCommon import *
from IPython.display import HTML
from itertools import product
from base64 import b64encode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 9.3 MB/s 
Cloning into 'pixUtils2'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 22 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (22/22), done.
Note: checking out '410e006ce8a65c4391540977a6db0d7a7791b1b8'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 410e006 initial commits
[youtube] lINIOau7MOQ: Downloading webpage
[youtube] lINIOau7MOQ: Downloadi

In [ ]:
def show_video(video_path):
    desPath = f'{dirname(video_path)}/smallSize.mp4'
    if not exists(desPath):
        !ffmpeg -y -i "{video_path}" -vf "scale=iw/2:ih/2" "{desPath}" > /dev/null 2>&1
    video_file = open(desPath, "r+b").read()
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
    return HTML(f"""<video controls><source src="{video_url}"></video>""")


def getData(sr, sc, histSize, desDir, imPath):
    img = cv2.imread(imPath)
    img = imResize(img, [sr, sc], interpolation='aa')
    tileHist = cv2.calcHist([img], [0, 1, 2], None, histSize, [0, 256, 0, 256, 0, 256]).ravel().astype('f4') / img.size
    tilePath = dirop(f"{desDir}/tiles/{filename(imPath)}.png")
    cv2.imwrite(tilePath, img)
    return tileHist, tilePath


def createTiles(sr, sc, histSize, nTileClusters, imPaths, desDir, smallClusterPercentileTh=.35):
    print("buliding tiles from images, this is one time job and it will take some time to complete")
    tik = clk()
    dirop(desDir, rm=' ')
    tileHists, tilePaths = [], []
    poolFn, poolArgs, poolIter = getData, [sr, sc, histSize, desDir], imPaths
    with PPool(os.cpu_count()) as pool:
        for tileHist, tilePath in tqdm(pool.imap_unordered(partial(poolFn, *poolArgs), poolIter), total=len(imPaths)):
            tileHists.append(tileHist)
            tilePaths.append(tilePath)
    tileHists = np.array(tileHists).reshape(len(tileHists), -1)
    print(f"nTileClusters: {nTileClusters}")
    tileCluster = MiniBatchKMeans(nTileClusters)
    tileCluster.partial_fit(tileHists)

    print(f"calculate cluster population")
    cluster2tilePaths = defaultdict(list)
    for y, tilePath in zip(tileCluster.labels_, tilePaths):
        cluster2tilePaths[y].append(tilePath)
    cluster2tilePaths = cluster2tilePaths.items()
    smallClusterTh = np.percentile([len(tPaths) for k, tPaths in cluster2tilePaths], smallClusterPercentileTh * 100)

    print(f"remove small cluster")
    okClusterIds = []
    for y, clusterTilePaths in sorted(cluster2tilePaths, key=lambda x: x[0]):
        if len(clusterTilePaths) > smallClusterTh:
            okClusterIds.append(y)
            for tilePath in clusterTilePaths:
                dirop(tilePath, symDir=f"{desDir}/ok/{y}")
        else:
            for tilePath in clusterTilePaths:
                dirop(tilePath, symDir=f"{desDir}/err/smallCluster/{y}")
    print(f"okClusterIds:[{len(okClusterIds)}] {okClusterIds}")

    tileCluster.cluster_centers_ = tileCluster.cluster_centers_[okClusterIds]

    tileCluster.paths = [list() for _ in tileCluster.cluster_centers_]
    res = []
    for y, tileHist, tilePath in zip(tileCluster.labels_, tileHists, tilePaths):
        if y in okClusterIds:
            newY = okClusterIds.index(y)
            tileCluster.paths[newY].append(tilePath)
            res.append(newY)
    tileCluster.labels_ = np.array(res)

    for y in rglob(f"{desDir}/ok/*"):
        newY = okClusterIds.index(int(basename(y)))
        dirop(y, symDir=f"{desDir}/tileClusters", desName=newY)
        writeBook(f"{desDir}/tileClusters/{newY}.txt", ', '.join([str(i) for i in tileCluster.cluster_centers_[newY]]))

    print("26 createTiles imageCollage2 tik.tok() ", tik.tok("").last())
    return tileCluster


def subImg2img(iLocs, sImgs, imgs, photoMosaic, sr, sc):
    batchSz = len(imgs)
    if not len(photoMosaic):
        photoMosaic = np.empty_like(imgs[0])
    if sImgs is None:
        for img, photoMosaic in zip(imgs, [photoMosaic] * batchSz):
            yield img, photoMosaic
    else:
        batchIxs = [list() for i in range(batchSz)]
        for batchIx, i, j, imgIx in iLocs:
            batchIxs[batchIx].append([i, j, imgIx])
        for img, iLocs in zip(imgs, batchIxs):
            for i, j, imgIx in iLocs:
                si, sj = i * sr, j * sc
                photoMosaic[si:si + sr, sj:sj + sc] = sImgs[imgIx]
            yield img, photoMosaic


def img2subImg(imgs, img, active, iLocs, hists, scaleUp, sr, sc, histSize):
    r, c = img.shape[:2]
    nRow, nCol = math.ceil(r * scaleUp / sr), math.ceil(c * scaleUp / sc)
    img = imResize(img, [nRow * sr, nCol * sc])
    active = imResize(active, [nRow * sr, nCol * sc])
    for i, j in product(range(nRow), range(nCol)):
        si, sj = i * sr, j * sc
        a = active[si:si + sr, sj:sj + sc].mean() > .5
        if a:
            s = img[si:si + sr, sj:sj + sc]
            hist = cv2.calcHist([s], [0, 1, 2], None, histSize, [0, 256, 0, 256, 0, 256]).ravel().astype('f4') / s.size
            iLocs.append([len(imgs), i, j, len(hists)])
            hists.append(hist)
    imgs.append(img)
    return iLocs, hists


def img2photoMosaicWithThread(imClusters, tileCluster, iLocs, tileHeight, tileWidth):
    label2tilePaths = defaultdict(list)
    for label, (batchIx, i, j, imgIx) in zip(imClusters.labels_, iLocs):
        label2tilePaths[label].append(imgIx)
    photoMosaic = np.empty([len(iLocs), tileHeight, tileWidth, 3], dtype='u1')
    poolFn, poolArgs, poolIter = fastImg2tile, [imClusters, tileCluster, photoMosaic], label2tilePaths.items()
    with TPool(os.cpu_count()) as pool:
        for _ in pool.imap_unordered(partial(poolFn, *poolArgs), poolIter):
            pass
    return photoMosaic


def fastImg2tile(imClusters, tileCluster, photoMosaic, labelILocs):
    label, iLocs = labelILocs
    center = imClusters.cluster_centers_[label][None]
    tileCenters = tileCluster.cluster_centers_.astype(np.double)
    # errs = np.linalg.norm(center - tileCenters, axis=1)
    errs = np.empty([1, len(tileCenters)], dtype=np.double)
    cdist_cosine_double_wrap(center.astype(np.double), tileCenters, errs)
    iBests = np.argmin(errs.ravel())
    iBest = iBests
    clusterPaths = tileCluster.paths[iBest]
    nPaths = len(clusterPaths)
    for ix, imgIx in enumerate(iLocs):
        img = cv2.imread(clusterPaths[ix % nPaths])
        img = img if random.random() < .5 else img[:, ::-1]
        photoMosaic[imgIx] = img


def getActiveLocs(img, pImg):
    if not len(pImg):
        actives = np.ones_like(img, dtype=bool)
        pImg = img.copy()
    else:
        actives = np.linalg.norm(img.astype('f4') - pImg.astype('f4'), axis=-1) > 16
        pImg[actives] = img[actives].copy()
    return np.uint8(actives), pImg


class ImCluster:
    def __init__(self, nImClusters, batchSz):
        self.batchSz = batchSz
        self.resetClusters = None
        self.nImClusters = nImClusters
        self.imClusters = MiniBatchKMeans(nImClusters)

    def prcs(self, hists):
        if self.resetClusters is None:
            self.resetClusters = len(hists) * .5
        if len(hists) > self.resetClusters:  # reset cluster for scene transition
            self.imClusters = MiniBatchKMeans(self.nImClusters)
        self.imClusters.partial_fit(hists)
        return self.imClusters

In [ ]:
def main(db, tileDir, vPath, outDir, tileHeight=32, tileWidth=32, nFeature=5, nImClusters=150, nTileClusters=500, imScaleUp=4, batchSz=256, deleteCache=False):
    outDir = dirop(f"{outDir}/{filename(vPath)}", rm=' ')
    outPath = f"{outDir}/videoMosaic.mp4"
    pName = f"tiles_{tileHeight}_{tileWidth}_{nFeature}_{nTileClusters}"
    histSize = np.array([nFeature, nFeature, nFeature], dtype=int)
    tileCluster = readPkl(f'{db}/{pName}.pkl', lambda: createTiles(tileHeight, tileWidth, histSize, nTileClusters, rglob(f'{tileDir}/*.jpg'), f'{db}/{pName}'), rm=deleteCache)
    prr(f"151 main imageCollage3 tileCluster.cluster_centers_", tileCluster.cluster_centers_, '')
    pImg, mosaicImg = np.array([]), np.array([])
    cam = cv2.VideoCapture(vPath)
    imClusters = ImCluster(nImClusters, batchSz)
    imgs, iLocs, hists = list(), list(), list()
    withoutAudio = f'{outDir}/withoutAudio.mp4'

    def __prcs(mosaicImg):
        sImgs = None
        if len(hists):
            clusters = imClusters.prcs(hists)
            sImgs = img2photoMosaicWithThread(clusters, tileCluster, iLocs, tileHeight, tileWidth)
        for img, mosaicImg in subImg2img(iLocs, sImgs, imgs, mosaicImg, tileHeight, tileWidth):
            alpha = .5
            disp = cv2.addWeighted(mosaicImg, alpha, img, 1 - alpha, 0)
            vWriter.write(disp)
            # showImg('imageCollage3_disp', disp, 1)
        return mosaicImg

    with VideoWriter(withoutAudio, cam) as vWriter:
        for fno, ftm, img in tqdm(videoPlayer(cam)):
            active, pImg = getActiveLocs(img, pImg)
            iLocs, hists = img2subImg(imgs, img, active, iLocs, hists, imScaleUp, tileHeight, tileWidth, histSize)
            if len(imgs) == batchSz:
                mosaicImg = __prcs(mosaicImg)
                imgs, iLocs, hists = list(), list(), list()
        if len(imgs):
            mosaicImg = __prcs(mosaicImg)
    exeIt(f'ffmpeg -i "{withoutAudio}" -i "{vPath}" -map 0:v -map 1:a -c:v copy -shortest -y "{outPath}" > /dev/null 2>&1')
    return outPath

mosaicPath = main('/content/db/imageCollage', '/content/db/img_align_celeba', rglob('/content/db/srcVideos/*.mp4')[0], '/content/db/desVideos')

executing: <function main.<locals>.<lambda> at 0x7ff96e9cd0e0>
buliding tiles from images, this is one time job and it will take some time to complete
100%|██████████| 202599/202599 [05:31<00:00, 611.57it/s]
nTileClusters: 500
calculate cluster population
remove small cluster
okClusterIds:[323] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 21, 22, 23, 24, 26, 27, 30, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 59, 60, 61, 62, 63, 64, 67, 68, 70, 71, 72, 73, 75, 78, 79, 80, 82, 84, 86, 87, 88, 89, 90, 91, 92, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 106, 107, 108, 110, 111, 112, 113, 114, 115, 116, 117, 119, 120, 121, 122, 123, 126, 127, 129, 130, 131, 133, 134, 136, 137, 139, 140, 142, 143, 145, 147, 148, 150, 151, 152, 154, 156, 158, 159, 162, 163, 164, 165, 166, 167, 169, 171, 172, 173, 174, 175, 176, 178, 179, 180, 181, 182, 183, 184, 186, 187, 189, 191, 192, 193, 194, 196, 198, 201, 203, 205, 207, 208, 209

In [ ]:
show_video(mosaicPath)

In [ ]:
# def randomMosaic(db, tileDir, vPath, tileHeight=24, tileWidth=24, nFeature=5, nImClusters=50, nTileClusters=500, imScaleUp=4, deleteCache=False):
#     cam = cv2.VideoCapture(vPath)
#     withoutAudio = f'{dirname(vPath)}/videoMosaic_withoutAudio_{basename(vPath)}'
#     with VideoWriter(withoutAudio, cam) as vWriter:
#         tilePaths = rglob('/home/ubuntu/otherExp/edaTicket/db/imageCollage/tiles_24_24_5_500/tileClusters/*/*.png')
#         scaleUp = imScaleUp
#         sr, sc = tileWidth, tileHeight
#         photoMosaic = np.array([])
#         for fno, ftm, img in tqdm(videoPlayer(cam, 0, 15)):
#             active, pImg = getActiveLocs(img, pImg)
#             r, c = img.shape[:2]
#             nRow, nCol = math.ceil(r * scaleUp / sr), math.ceil(c * scaleUp / sc)
#             img = imResize(img, [nRow * sr, nCol * sc])
#             active = imResize(active, [nRow * sr, nCol * sc])
#             if not len(photoMosaic):
#                 photoMosaic = np.zeros_like(img)
#             for i, j in product(range(nRow), range(nCol)):
#                 si, sj = i * sr, j * sc
#                 a = active[si:si + sr, sj:sj + sc].mean() > .5
#                 if a:
#                     photoMosaic[si:si + sr, sj:sj + sc] = cv2.imread(random.choice(tilePaths))
#             alpha = .5
#             disp = cv2.addWeighted(photoMosaic, alpha, img, 1 - alpha, 0)
#             vWriter.write(disp)